<a href="https://colab.research.google.com/github/mshinohar/langchain-book/blob/main/langchain_ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. LangChain解説

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your api key"

## 4-1 LangChainの概要

In [ ]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


## 4-2 Language models

### LLMs

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

result = llm("自己紹介してください。")
print(result)



はじめまして、私は○○と申します。現在、○○大学で学んでいます。趣味は料理や旅行などです。特に料理は好きで、毎日新しいレシピを試しています。今までに海外旅行も何回かしていて、異文化に触れるのが大好きです。今後も新しいことに挑戦していきたいと思っています。よろしくお願いします。


### Chat models

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = chat(messages)
print(result.content)

はい、先ほどおっしゃった通り、あなたの名前はジョンですね。何か他にお手伝いできることはありますか？


### Callbackを使ったストリーミング

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

messages = [HumanMessage(content="自己紹介してください")]
result = chat(messages)

はい、私はオープンAIの言語モデルであるGPT-3です。私は自然言語処理の能力を持ち、様々なトピックについての質問や会話に応えることができます。私は大量のデータを学習しており、文章の生成や意味の理解、文脈に基づいた回答などを行うことができます。私の目標は、人々がより簡単に情報を得たり、問題を解決したりするのを支援することです。どのようにお手伝いできますか？

## 4-3 Prompts

### Prompt templates

In [ ]:
from langchain.prompts import PromptTemplate

template = """
以下の料理のレシピを考えてください。

料理名: {dish}
"""

prompt = PromptTemplate(
   input_variables=["dish"],
   template=template,
)

result = prompt.format(dish="カレー")
print(result)


以下の料理のレシピを考えてください。

料理名: カレー



### ChatPromptTemplate

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたは{country}料理のプロフェッショナルです。"),
    HumanMessagePromptTemplate.from_template("以下の料理のレシピを考えてください。\n\n料理名: {dish}")
])

messages = chat_prompt.format_prompt(country="イギリス", dish="肉じゃが").to_messages()

print(messages)

[SystemMessage(content='あなたはイギリス料理のプロフェッショナルです。'), HumanMessage(content='以下の料理のレシピを考えてください。\n\n料理名: 肉じゃが')]


In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
result = chat(messages)
print(result.content)

材料:
- 牛肉 (500g)
- じゃがいも (4個)
- にんじん (1本)
- 玉ねぎ (1個)
- にんにく (2片)
- トマト缶 (1缶)
- しょうゆ (大さじ2)
- 砂糖 (大さじ1)
- ケチャップ (大さじ2)
- ベジタブルストック (200ml)
- オリーブオイル (大さじ2)
- 塩 (お好みで)
- 黒こしょう (お好みで)
- パセリ (お好みで)

作り方:
1. じゃがいも、にんじん、玉ねぎを一口サイズに切ります。
2. 牛肉を食べやすい大きさに切ります。
3. にんにくをみじん切りにします。
4. 大きめの鍋にオリーブオイルを熱し、にんにくを炒めます。
5. 牛肉を加えて炒め、全体に焼き色がつくまで炒めます。
6. じゃがいも、にんじん、玉ねぎを加えてさらに炒めます。
7. トマト缶、しょうゆ、砂糖、ケチャップ、ベジタブルストックを加え、全体がひたひたになるまで水を足します。
8. 弱火にして、蓋をして約30分間煮込みます。途中で塩や黒こしょうで味を調えます。
9. 煮込みが終わったら、パセリを散らして完成です。

肉じゃがはご飯やパンと一緒に食べると美味しいです。お好みでトッピングやアレンジを加えても楽しめます。


## 4-4 Output Parsers


### PydanticOutputParser

In [ ]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

In [ ]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
format_instructions = parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"title": "Ingredients", "description": "ingredients of the dish", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "steps to make the dish", "type": "array", "items": {"type": "string"}}}, "required": ["ingredients", "steps"]}
```


In [ ]:
from langchain.prompts import PromptTemplate

template = """料理のレシピを考えてください。

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": format_instructions}
)

In [ ]:
formatted_prompt = prompt.format(dish="カレー")

print(formatted_prompt)

料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"title": "Ingredients", "description": "ingredients of the dish", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "steps to make the dish", "type": "array", "items": {"type": "string"}}}, "required": ["ingredients", "steps"]}
```

料理名: カレー



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
messages = [HumanMessage(content=formatted_prompt)]
output = chat(messages)

print(output.content)

{
  "ingredients": [
    "玉ねぎ",
    "にんじん",
    "じゃがいも",
    "豚肉",
    "カレールー",
    "水"
  ],
  "steps": [
    "玉ねぎ、にんじん、じゃがいもを切る",
    "豚肉を炒める",
    "野菜を加えて炒める",
    "水を加えて煮込む",
    "カレールーを加えて溶かす",
    "煮込んで完成"
  ]
}


In [ ]:
recipe = parser.parse(output.content)
print(recipe)

ingredients=['玉ねぎ', 'にんじん', 'じゃがいも', '豚肉', 'カレールー', '水'] steps=['玉ねぎ、にんじん、じゃがいもを切る', '豚肉を炒める', '野菜を加えて炒める', '水を加えて煮込む', 'カレールーを加えて溶かす', '煮込んで完成']


## 4-4 Chains

### LLMChain―PromptTemplate・Language model・OutputParserをつなぐ

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

template = """料理のレシピを考えてください。

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain import LLMChain

chain = LLMChain(prompt=prompt, llm=chat, output_parser=output_parser)

recipe = chain.run(dish="カレー")

print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['玉ねぎ', 'にんじん', 'じゃがいも', '豚肉', 'カレールー', '水'] steps=['玉ねぎ、にんじん、じゃがいもを切る', '豚肉を炒める', '野菜を加えて炒める', '水を加えて煮込む', 'カレールーを加えて溶かす', '煮込んで完成']


### SimpleSequentialChain―ChainとChainをつなぐ

In [ ]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

cot_template = """以下の質問に回答してください。

質問: {question}

ステップバイステップで考えましょう。
"""

cot_prompt = PromptTemplate(
   input_variables=["question"],
   template=cot_template,
)

cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

In [ ]:
summarize_template = """以下の文章を結論だけ一言に要約してください。

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

In [ ]:
from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])

result = cot_summarize_chain(
   "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？"
)
print(result["output"])

残りのリンゴは10個です。


上記の入力は [Chain-of-Thoughtプロンプティング | Prompt Engineering Guide](https://www.promptingguide.ai/jp/techniques/cot) から引用しました。

## 4-5 Memory

### ConversationBufferMemory

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

#chat = ChatOpenAI(model_name="gpt-4", temperature=0)
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    ai_message = conversation.run(input=user_message)
    print(f"AI: {ai_message}")

You: Hello, I'm Masato.
AI: Hello Masato! How can I assist you today?
You: Do you know my name?
AI: Yes, your name is Masato. You mentioned it at the beginning of our conversation.


KeyboardInterrupt: ignored

### （コラム）Chat modelsでMemoryを使う場合の注意

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = chat(messages)
print(result.content)

はい、先ほどおっしゃった通り、あなたの名前はジョンですよ。何か他にお手伝いできることはありますか？
